In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import math
from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2, ToTensor

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

DIR_INPUT = '/kaggle/input/global-wheat-detection/'
DIR_TRAIN = '/kaggle/input/global-wheat-detection/train'
DIR_TEST = '/kaggle/input/global-wheat-detection/test'

In [ ]:
import copy
import json

import numpy as np
import torch
import albumentations as A

In [ ]:
train_df = pd.read_csv(DIR_INPUT+'train.csv')
train_df.shape

train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)
train_df['x'] = train_df['x'].astype(np.float)
train_df['y'] = train_df['y'].astype(np.float)
train_df['w'] = train_df['w'].astype(np.float)
train_df['h'] = train_df['h'].astype(np.float)

In [ ]:
print(train_df)

In [ ]:
from sklearn.utils import shuffle
import random
from tqdm.auto import tqdm

In [ ]:
df = train_df
image_ids = df['image_id'].unique()
print(image_ids)
image_ids = shuffle(image_ids)
print(image_ids)

In [ ]:
labels = [np.zeros((0, 5), dtype=np.float32)] * len(image_ids)
print(labels)

In [ ]:
img_size = 1024
im_w = 1024
im_h = 1024
for i, img_id in enumerate(tqdm(image_ids)):
    records = df[df['image_id'] == img_id]
    boxes = records[['x', 'y', 'w', 'h']].values
    boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
    boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
    boxesyolo = []
    for box in boxes:
        x1, y1, x2, y2 = box
        xc, yc, w, h = 0.5*x1/im_w+0.5*x2/im_w, 0.5*y1/im_h+0.5*y2/im_h, abs(x2/im_w-x1/im_w), abs(y2/im_h-y1/im_h)
        boxesyolo.append([0, xc, yc, w, h])
    labels[i] = np.array(boxesyolo)

In [ ]:
print(labels)

In [ ]:
def load_image(index):
    # loads 1 image from dataset, returns img, original hw, resized hw
    image_id = image_ids[index]
    imgpath = DIR_TRAIN
    img = cv2.imread(f'{imgpath}/{image_id}.jpg', cv2.IMREAD_COLOR)
    
    assert img is not None, 'Image Not Found ' + imgpath
    h0, w0 = img.shape[:2]  # orig hw
    return img, (h0, w0), img.shape[:2]

In [ ]:
def random_affine(img, targets=(), degrees=10, translate=.1, scale=.1, shear=10, border=0):
    # torchvision.transforms.RandomAffine(degrees=(-10, 10), translate=(.1, .1), scale=(.9, 1.1), shear=(-10, 10))
    # https://medium.com/uruvideo/dataset-augmentation-with-random-homographies-a8f4b44830d4

    if targets is None:  # targets = [cls, xyxy]
        targets = []
    height = img.shape[0] + border * 2
    width = img.shape[1] + border * 2

    # Rotation and Scale
    R = np.eye(3)
    a = random.uniform(-degrees, degrees)
    # a += random.choice([-180, -90, 0, 90])  # add 90deg rotations to small rotations
    s = random.uniform(1 - scale, 1 + scale)
    R[:2] = cv2.getRotationMatrix2D(angle=a, center=(img.shape[1] / 2, img.shape[0] / 2), scale=s)

    # Translation
    T = np.eye(3)
    T[0, 2] = random.uniform(-translate, translate) * img.shape[0] + border  # x translation (pixels)
    T[1, 2] = random.uniform(-translate, translate) * img.shape[1] + border  # y translation (pixels)

    # Shear
    S = np.eye(3)
    S[0, 1] = math.tan(random.uniform(-shear, shear) * math.pi / 180)  # x shear (deg)
    S[1, 0] = math.tan(random.uniform(-shear, shear) * math.pi / 180)  # y shear (deg)

    # Combined rotation matrix
    M = S @ T @ R  # ORDER IS IMPORTANT HERE!!
    if (border != 0) or (M != np.eye(3)).any():  # image changed
        img = cv2.warpAffine(img, M[:2], dsize=(width, height), flags=cv2.INTER_LINEAR, borderValue=(114, 114, 114))

    # Transform label coordinates
    n = len(targets)
    if n:
        # warp points
        xy = np.ones((n * 4, 3))
        xy[:, :2] = targets[:, [1, 2, 3, 4, 1, 4, 3, 2]].reshape(n * 4, 2)  # x1y1, x2y2, x1y2, x2y1
        xy = (xy @ M.T)[:, :2].reshape(n, 8)

        # create new boxes
        x = xy[:, [0, 2, 4, 6]]
        y = xy[:, [1, 3, 5, 7]]
        xy = np.concatenate((x.min(1), y.min(1), x.max(1), y.max(1))).reshape(4, n).T

        # # apply angle-based reduction of bounding boxes
        # radians = a * math.pi / 180
        # reduction = max(abs(math.sin(radians)), abs(math.cos(radians))) ** 0.5
        # x = (xy[:, 2] + xy[:, 0]) / 2
        # y = (xy[:, 3] + xy[:, 1]) / 2
        # w = (xy[:, 2] - xy[:, 0]) * reduction
        # h = (xy[:, 3] - xy[:, 1]) * reduction
        # xy = np.concatenate((x - w / 2, y - h / 2, x + w / 2, y + h / 2)).reshape(4, n).T

        # reject warped points outside of image
        xy[:, [0, 2]] = xy[:, [0, 2]].clip(0, width)
        xy[:, [1, 3]] = xy[:, [1, 3]].clip(0, height)
        w = xy[:, 2] - xy[:, 0]
        h = xy[:, 3] - xy[:, 1]
        area = w * h
        area0 = (targets[:, 3] - targets[:, 1]) * (targets[:, 4] - targets[:, 2])
        ar = np.maximum(w / (h + 1e-16), h / (w + 1e-16))  # aspect ratio
        i = (w > 4) & (h > 4) & (area / (area0 * s + 1e-16) > 0.2) & (ar < 10)

        targets = targets[i]
        targets[:, 1:5] = xy[i]

    return img, targets

In [ ]:
labels4 = []
index=10
s = img_size
xc, yc = [int(random.uniform(s * 0.5, s * 1.5)) for _ in range(2)]
print(xc)
print(yc)
indices = [index] + [random.randint(0, len(labels) - 1) for _ in range(3)]
print(indices)
for i, index in enumerate(indices):
    img, _, (h, w) = load_image(index)
    if i == 0:  # top left
            img4 = np.full((s * 2, s * 2, img.shape[2]), 114, dtype=np.uint8)  # base image with 4 tiles
            print(img4.shape)
            x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
            print("large",x1a,y1a,x2a,y2a)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            print("small",x1b,y1b,x2b,y2b)
    elif i == 1:  # top right
            x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
            print("large1",x1a,y1a,x2a,y2a)
            x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            print("small1",x1b,y1b,x2b,y2b)
    elif i == 2:  # bottom left
            x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
            print("large2",x1a,y1a,x2a,y2a)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            print("small2",x1b,y1b,x2b,y2b)
    elif i == 3:  # bottom right
            x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
            print("large3",x1a,y1a,x2a,y2a)
            x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            print("small3",x1b,y1b,x2b,y2b)
    img4[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]
    padw = x1a - x1b
    print("padw",padw)
    padh = y1a - y1b
    print("padh",padh)
    x = labels[index]
    print("XXXXXXXX")
    print(x)
    d = x.copy()
    if x.size > 0:  # Normalized xywh to pixel xyxy format
        d[:, 1] = w * (x[:, 1] - x[:, 3] / 2) + padw
        d[:, 2] = h * (x[:, 2] - x[:, 4] / 2) + padh
        d[:, 3] = w * (x[:, 1] + x[:, 3] / 2) + padw
        d[:, 4] = h * (x[:, 2] + x[:, 4] / 2) + padh
    labels4.append(d)
if len(labels4):
    labels4 = np.concatenate(labels4, 0)
    # np.clip(labels4[:, 1:] - s / 2, 0, s, out=labels4[:, 1:])  # use with center crop
    np.clip(labels4[:, 1:], 0, 2 * s, out=labels4[:, 1:])

img3, labels4 = random_affine(img4, labels4,
                              degrees=1.98 * 2,
                              translate=0.05 * 2,
                              scale=0.05 * 2,
                              shear=0.641 * 2,
                              border=-s // 2)

In [ ]:
print(labels4[:,:])

In [ ]:
import cv2
import matplotlib.pyplot as plt
plt.imshow(img4)
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt
plt.imshow(img3)
plt.show()